# 그냥 한번 뉴럴네트웤 테스트...ㅎ
* 데이터는 아래 데이터와, 그리고 각 사람당 리뷰한 맥주 데이터의 평균을 임베딩하여 추가함
* 아니 최신버전 쓰고 있었는데 갑자기 1.3.1로 버전 업그레이드 됨;;; 다꼬임 아오
  * 에러는 안뜨니까 일단 그냥 써야겠다.

In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg
import numpy as np

from joblib import load, dump

### 데이터 설명
* review 테이블 : 사용자가 맥주에 준 점수, 즉 y값 하나만을 위해 가져옴
* beer 테이블 : 맥주 데이터
* member 테이블 : 멤버 나이, 성별
* member_mean 테이블 : 사용자의 현재까지 기록의 평균

In [2]:
conn = psycopg.connect(host="sulleong.site", dbname="sulleong", user="sulleong", password="Sulleong104**", port=5432)
sql = '''SELECT b.abv AS beer_abv, b.brewer_id, b.country, b.large_category, b.sub_category, m.age, m.gender, ms.*, r.overall AS y
    FROM member AS m
    INNER JOIN review AS r ON r.member_id = m.id
    INNER JOIN beer AS b ON r.beer_id = b.id
    INNER JOIN learning_dataset AS ms ON r.member_id = ms.member_id
    WHERE b.country IS NOT NULL AND b.id IN (1, 2, 3);
    '''  # WHERE b.id IN (1, 2, 3);
reviews = sqlio.read_sql_query(sql, conn)
conn.close()

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_4956\1815149751.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  reviews = sqlio.read_sql_query(sql, conn)


In [3]:
# 불러온 데이터, age랑 gender는 못써먹어서 일단 제외
reviews = reviews.drop(columns=['member_id', 'age', 'gender'])

# brewery가 너무 많아서 안될듯 ㅠㅠㅠ
reviews = reviews.drop(columns='brewer_id')

In [4]:
# 나누는것은 파이썬에서 해야하나...?ㅠㅠ
review_notdiv = reviews[['beer_abv', 'country', 'large_category', 'sub_category', 'y']]
review_todiv = reviews.drop(columns=['beer_abv', 'country', 'large_category', 'sub_category', 'y'])

In [5]:
review_todiv = review_todiv.div(review_todiv['divide_size'], axis=0)
review_todiv = review_todiv.drop(columns='divide_size')

In [6]:
reviews = pd.concat([review_notdiv, review_todiv], axis=1)

In [7]:
reviews

,beer_abv,country,large_category,sub_category,y,overall,abv,review_count,country_AR,country_AT,...,sub_category_Specialty Grain,sub_category_Spice/Herb/Vegetable,sub_category_Stout,sub_category_Strong Pale Lager/Imperial Pils,sub_category_Sweet Stout,sub_category_Traditional Ale,sub_category_Vienna,sub_category_Weizen Bock,sub_category_Wheat Ale,sub_category_Zwickel/Keller/Landbier
0,8.0,US,Wheat Beer,Heller Bock,12,15.058405,8.043746,575.414530,0.000000,0.000000,...,0.001425,0.027066,0.038462,0.011396,0.021368,0.005698,0.000000,0.000000,0.007123,0.000000
1,8.0,US,Wheat Beer,Heller Bock,10,14.068508,6.500495,245.788696,0.000000,0.000428,...,0.007707,0.025262,0.017769,0.005780,0.015414,0.006423,0.008135,0.006209,0.011133,0.007707
2,8.0,US,Wheat Beer,Heller Bock,14,13.456952,6.419069,299.377232,0.000000,0.000319,...,0.005421,0.022321,0.031569,0.003827,0.031250,0.005740,0.008291,0.005102,0.014987,0.004464
3,8.0,US,Wheat Beer,Heller Bock,12,14.014211,7.538923,576.936425,0.000748,0.000748,...,0.006731,0.038893,0.017203,0.009723,0.012715,0.005984,0.007479,0.008227,0.008975,0.000748
4,8.0,US,Wheat Beer,Heller Bock,10,14.403846,6.864904,562.427885,0.000000,0.000000,...,0.004808,0.024038,0.033654,0.004808,0.062500,0.004808,0.004808,0.009615,0.009615,0.004808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,5.0,US,Lagers,Vienna,8,13.497976,6.994381,526.605668,0.000000,0.000810,...,0.008907,0.023482,0.027530,0.005668,0.011336,0.004858,0.008907,0.003239,0.010526,0.001619
2046,5.0,US,Lagers,Vienna,10,14.329204,7.850761,621.171681,0.000000,0.001770,...,0.001770,0.010619,0.015929,0.000000,0.015929,0.001770,0.007080,0.003540,0.017699,0.000000
2047,5.0,US,Lagers,Vienna,14,13.813014,6.423837,435.163052,0.000748,0.000000,...,0.009723,0.029170,0.017203,0.008975,0.014211,0.005236,0.011219,0.008227,0.012715,0.004488
2048,5.0,US,Lagers,Vienna,9,15.368421,7.965057,669.560641,0.000000,0.000000,...,0.002288,0.025172,0.025172,0.006865,0.009153,0.013730,0.002288,0.000000,0.006865,0.000000


## 전처리
* 가져온 데이터 전처리 함
* 일단 무식하게 다 바꾸는 식으로 했는데
  * 이제 beer_id에 따라 범주화 후 그 데이터를 사용자리뷰와 join시키는 식으로 활용해야 할 듯

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# 인코더 설정, none값은 그냥 0 박음
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', dtype=int)

# 숫자형 데이터
numeric_data = reviews.drop(columns=['large_category', 'sub_category', 'country'])

# 범주형 데이터 맞추고 변환
encoder.fit(reviews[['large_category', 'sub_category', 'country']])
encoding_beer = encoder.transform(reviews[['large_category', 'sub_category', 'country']])
# # 다시 pandas 데이터프레임으로 변환해서 붙일수 있도록 만듬
encoded_categories_beer = pd.DataFrame(encoding_beer, columns=encoder.get_feature_names_out(input_features=['large_category', 'sub_category', 'country']))
# print(processed_beer_notscaled)
reviews = pd.concat([numeric_data, encoded_categories_beer], axis=1)

In [8]:
# joblib 파일 있을때 전처리하기
numeric_data = reviews.drop(columns=['large_category', 'sub_category', 'country'])
encoder = load('./beer_encoder.joblib')
encoding_beer = encoder.transform(reviews[['large_category', 'sub_category', 'country']])
encoded_categories_beer = pd.DataFrame(encoding_beer, columns=encoder.get_feature_names_out(input_features=['large_category', 'sub_category', 'country']))
reviews = pd.concat([numeric_data, encoded_categories_beer], axis=1)

c:\Users\SSAFY\Desktop\realpjt\S09P22A104\recommend\venv\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.3.0 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
reviews

,beer_abv,y,overall,abv,review_count,country_AR,country_AT,country_AU,country_BE,country_CA,...,country_KR,country_MX,country_NL,country_NZ,country_PL,country_SE,country_SG,country_TH,country_TR,country_US
0,8.0,12,15.058405,8.043746,575.414530,0.000000,0.000000,0.000000,0.024217,0.008547,...,0,0,0,0,0,0,0,0,0,1
1,8.0,10,14.068508,6.500495,245.788696,0.000000,0.000428,0.000214,0.013487,0.005352,...,0,0,0,0,0,0,0,0,0,1
2,8.0,14,13.456952,6.419069,299.377232,0.000000,0.000319,0.000638,0.012755,0.006696,...,0,0,0,0,0,0,0,0,0,1
3,8.0,12,14.014211,7.538923,576.936425,0.000748,0.000748,0.000748,0.043381,0.014211,...,0,0,0,0,0,0,0,0,0,1
4,8.0,10,14.403846,6.864904,562.427885,0.000000,0.000000,0.000000,0.038462,0.009615,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,5.0,8,13.497976,6.994381,526.605668,0.000000,0.000810,0.000000,0.025911,0.008907,...,0,0,0,0,0,0,0,0,0,1
2046,5.0,10,14.329204,7.850761,621.171681,0.000000,0.001770,0.000000,0.033628,0.003540,...,0,0,0,0,0,0,0,0,0,1
2047,5.0,14,13.813014,6.423837,435.163052,0.000748,0.000000,0.000748,0.029170,0.011967,...,0,0,0,0,0,0,0,0,0,1
2048,5.0,9,15.368421,7.965057,669.560641,0.000000,0.000000,0.000000,0.018307,0.009153,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
dump(encoder, './beer_encoder.joblib')

In [ ]:
# abv에 null값 있음;;; 처리해야 함, 그냥 평균 넣음
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(reviews['beer_abv'].values.reshape(-1, 1))
reviews['beer_abv'] = imp.transform(reviews['beer_abv'].values.reshape(-1, 1))

In [ ]:
reviews

In [10]:
y = reviews['y']
X = reviews.drop(columns=['y'])

In [11]:
# 데이터 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
dump(scaler, './model_scaler.joblib')

In [ ]:
np.where(np.isnan(X))

## 전처리가 모두 끝나면 regressor로 예측함

In [ ]:
from sklearn.neural_network import MLPRegressor

# solver = ’adam’
# alpha = 0.0001
# batch_size = 200
# learning_rate = 0.001
# loss function = square error, 이것만 지원한다고 함
# output에는 activation function 없음

regr = MLPRegressor(max_iter=250, hidden_layer_sizes=(50,50,30)).fit(X, y)

In [12]:
# dump(regr, './model_regressor.joblib')
regr = load('./model_regressor.joblib')

c:\Users\SSAFY\Desktop\realpjt\S09P22A104\recommend\venv\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator MLPRegressor from version 1.3.0 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
regr.n_features_in_

234

In [14]:
regr.predict(X[:15])

array([14.16661479, 13.90168719, 13.05325517, 14.39912893, 14.7242885 ,
       13.06767952, 14.8152725 , 14.10075786, 13.35827761, 14.13042267,
       13.94617937, 13.54188793, 14.34140561, 13.55867431, 13.597444  ])

In [15]:
y[:10]

0    12
1    10
2    14
3    12
4    10
5     3
6    13
7     8
8    11
9    12
Name: y, dtype: int64